# OpenAI Whisper Demo

taken from:
[Whisper Github](https://github.com/openai/whisper#python-usage)

In [6]:
# Imports
import os
from pathlib import Path
import whisper

In [8]:
# define paths

cwd = os.getcwd()
p = Path(cwd)
notebook_dir = p.parent
root_dir = notebook_dir.parent.parent
data_dir = os.path.join(root_dir, "data")
external_dir = os.path.join(data_dir, "external")

mp3_file = os.path.join(external_dir, "Laughter_Lift.mp3")
print(f"mp3_file exists: {os.path.exists(mp3_file)}")

mp3_file exists: True


In [11]:
model = whisper.load_model("base")

In [12]:
base_result = model.transcribe(mp3_file, fp16=False)
print(base_result["text"])

 More with Emily Watson intake two adds in a minute, but first it's time once again very very good news everybody We step into our love to lift as are Shazam here we go Hey mark singers you love the noble gases joke so much last week I was gonna tell you another one, but all the good ones are gone I got that I got that because argons are gas it is is no gas. I got it. Okay. Anyway. Here's another science you want for you I was out at a pub with rooms in showbiz north slendon on Saturday for a wedding reception a neutron walked in How much for a pint? He said for you no charge said the barman and I get that as well because in neutron has no charge Cyber atomic particle with no charge and then a photon walked in. I'd like a room for the night. Please. He said Certainly do you have any luggage we can take up to the room ask the receptionist no said the photon. I'm traveling light Because the photon is like a little like particle quantum of light In it was in fact it's an education today. 

In [9]:

md_model = whisper.load_model("medium")
md_result = md_model.transcribe(mp3_file, fp16=False)
print(md_result["text"])

 More with Emily Watson in take two, ads in a minute. But first, it's time once again, very, very good news everybody, we step into our laughter lift. Huzzah. Shazam. Here we go. Hey Mark, seeing as you loved the noble gases joke so much last week, I was going to tell you another one, but all the good ones are gone. I got that, I got that, because argon's a gas. It is, it's a noble gas. I got it, okay. Anyway, here's another science you want for you, are you ready? I was out at a pub with rooms in Chobhub's North London on Saturday for a wedding reception. A neutron walked in. How much for a pint, he said. For you, no charge, said the barman. And I get that as well, because a neutron has no charge. Subatomic particle with no charge. And then a photon walked in. I'd like a room for the night, please, he said. Certainly, do you have any luggage we can take up to the room? Asked the receptionist. No, said the photon. I'm traveling light. Because a photon is a... It is a light particle. Qu

## Findings

`base` model is not satisfactory. The punctuation and sentences are problematic and a number of words are wrong.
`medium` model is much much better. It seems to have correct sentences/punctuation. Most of the problems in `base` are resolved. It even got the French name Noémie correct. One thing `medium` got wrong that `base` got right was "showbiz north London" as opposed to "Chobhub north London". Another thing `medium` got wrong was "Vanguard Easter" instead of "vanguardista". But this isn't a common term so is forgivable. I think both of these examples (and other Wittertainment jargon) could be improved with model fine-tuning.